In [22]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib
import yfinance as yf

In [4]:
previous_vol_df = pd.read_csv('~/Code/giadapi/crypto/raw_data/all_columns_daily_090323.csv',lineterminator='\n',index_col=0)
previous_vol_df

,date,negative_bert,neutral_bert,positive_bert,daily_tweets,MA7_Sentiment,MA20_Sentiment,MA50_Sentiment,BTC_price
0,2021-01-01,0.083330,0.493347,0.423323,12561.859509,NaN,NaN,NaN,29374.152344
1,2021-01-02,0.134531,0.438811,0.426658,30124.352145,NaN,NaN,NaN,32127.267578
2,2021-01-03,0.147802,0.428930,0.423267,26167.331282,NaN,NaN,NaN,32782.023438
3,2021-01-04,0.104159,0.507274,0.388567,18041.199084,NaN,NaN,NaN,31971.914062
4,2021-01-05,0.081743,0.521464,0.396793,17473.551568,NaN,NaN,NaN,33992.429688
...,...,...,...,...,...,...,...,...,...
793,2023-03-05,0.107813,0.541465,0.350722,14968.952910,0.358301,0.358391,0.363660,22435.513672
794,2023-03-06,0.109010,0.507571,0.383419,137824.833122,0.359351,0.359776,0.363419,22429.757812
795,2023-03-07,0.123972,0.520640,0.355388,19028.050010,0.360315,0.359762,0.363403,22219.769531
796,2023-03-08,0.087879,0.515427,0.396694,19216.616481,0.366446,0.360095,0.364243,21718.080078


In [10]:
new_vol_df = pd.read

In [16]:
tweet_data = pd.read_csv('~/Code/giadapi/crypto/raw_data/cleaned_tweets_without_dupes_120323.csv',lineterminator='\n', index_col=0)
no_ninth = tweet_data[tweet_data["date"] != "2023-03-09"]

In [19]:
new_tweet_data = pd.read_csv('last_6_days.csv',lineterminator='\n', index_col=0)
fully_combined = pd.concat([no_ninth,new_tweet_data])

In [20]:
fully_combined

,datetime,username,text,date,negative_bert,neutral_bert,positive_bert
0,2021-01-01 22:09:59+00:00,sivilianlife,What's the chances @Grayscale is hedging short...,2021-01-01,0.002040,0.951312,0.046648
1,2021-01-01 22:09:39+00:00,LudicrousSpeed9,@CNBC Buy #Bitcoin to hedge against the result...,2021-01-01,0.004157,0.400750,0.595093
2,2021-01-01 22:09:35+00:00,SergioBowers1,@APompliano @pepsi @MonsterEnergy $DEO $KO e...,2021-01-01,0.000248,0.582107,0.417644
3,2021-01-01 22:09:21+00:00,bitvestor,Closed another chunk of this long now. Remaind...,2021-01-01,0.000974,0.807066,0.191960
4,2021-01-01 22:09:18+00:00,FROMKAIWITHLOVE,When you dig deeper into #kardiachain company ...,2021-01-01,0.000056,0.223449,0.776495
...,...,...,...,...,...,...,...
2513,2023-03-14 22:00:34+00:00,antoniomotgg,"""I love the smell of Napalm in the morning!"" -...",2023-03-14,0.018969,0.422259,0.558773
2514,2023-03-14 22:00:33+00:00,ashleygrilo12,They are about to turn the money printers on a...,2023-03-14,0.000570,0.312646,0.686783
2515,2023-03-14 22:00:30+00:00,BtcPulse,BTC hourly update $24658.30 | -0.54%📉 $B...,2023-03-14,0.368925,0.523527,0.107548
2516,2023-03-14 22:00:29+00:00,Bcubeai,#Bitcoin rallied 10% in one day. The next ones...,2023-03-14,0.000383,0.454934,0.544683


In [35]:
# Concern of order of data too


# Get daily tweets from tweet_data
def tweet_daily_rate(tweet_data_original):
    tweet_data = tweet_data_original.copy()
    
    tweet_data['datetime'] = pd.to_datetime(tweet_data['datetime'])
    tweet_data['hour'] = tweet_data['datetime'].dt.hour
    tweet_data['date'] = tweet_data['datetime'].dt.date
    
    daily_tweets = tweet_data.groupby(['date', 'hour']).agg({'negative_bert':'mean',
                                                 'neutral_bert':'mean',
                                                 'positive_bert':'mean',
                                                 'text':'count',
                                                 'datetime': lambda x: x.max() - x.min()})
    
    
    
    daily_tweets['datetime_seconds'] = daily_tweets['datetime'].dt.total_seconds()
    daily_tweets['tweets_rate'] = daily_tweets['text']/(daily_tweets['datetime_seconds'])
    daily_tweets_col = daily_tweets.groupby('date').agg({'tweets_rate':'mean'})
    daily_tweets_col['tweets_rate']*86400
    
    cut_off =daily_tweets.columns.get_loc("positive_bert")
    vol_data = daily_tweets.iloc[:,:cut_off+1]

    vol_data.reset_index(inplace=True)
    vol_data.drop('hour',axis=1,inplace=True)

    vol_data = vol_data.groupby(['date']).agg({'negative_bert':'mean',
                                        'neutral_bert':'mean',
                                        'positive_bert':'mean',
                                        })
    
    daily_tweets_col = daily_tweets.groupby('date').agg({'tweets_rate':'mean'})
    vol_data['daily_tweets'] = daily_tweets_col['tweets_rate']*86400
    vol_data.reset_index(inplace=True)
    return vol_data

# Pull btc price
def load_historic_data(symbol, start_date_str, end_date_str, period, interval):
    try:
        df = yf.download(symbol, start=start_date_str, end=end_date_str, period=period, interval=interval)
        #  Add symbol
    
        df["Symbol"] = symbol
        df['price_change'] = df['Close'].pct_change()
#         print(df)
        return df
    
    except:
        print(f'Error loading stock data for + {symbol}')
        return None

# Convert tweet_data to vol_data (Adding moving averages,btc)
# Automate saving of the file?
def daily_data(tweet_data, filename, start_date_str=None,symbol='BTC-USD', end_date_str=None, period='1d', interval='1d'):
    
    vol_data = tweet_daily_rate(tweet_data)
    
    if start_date_str == None:
        v_date = vol_data.columns.get_loc("date")
        start_date_str = vol_data.iloc[0,v_date]
    
    vol_data['MA7_Sentiment'] = vol_data['positive_bert'].rolling(window=8).mean()
    vol_data['MA20_Sentiment'] = vol_data['positive_bert'].rolling(window=21).mean()
    vol_data['MA50_Sentiment'] = vol_data['positive_bert'].rolling(window=51).mean()
    
    if end_date_str == None:
        today = datetime.date.today()
        today_str = datetime.datetime.strftime(today, "%Y-%m-%d")
        BTC_price = pd.DataFrame(load_historic_data(symbol, start_date_str, today_str, period, interval))
    else:
        BTC_price = pd.DataFrame(load_historic_data(symbol, start_date_str, end_date_str, period, interval))                    
#     print(BTC_price)
    vol_data['BTC_price']=BTC_price.reset_index()['Adj Close']
    vol_data['BTC_trading_volume']=BTC_price.reset_index()['Volume']
    
    vol_data = vol_data[['date','negative_bert', 'neutral_bert', 'positive_bert',
       'daily_tweets', 'MA7_Sentiment', 'MA20_Sentiment',
       'MA50_Sentiment', 'BTC_price','BTC_trading_volume']]
    
    
    vol_data.to_csv(f'{filename}.csv')
    return vol_data

In [36]:
new_vol_data = daily_data(fully_combined, 'new_vol_data_master_14-09', start_date_str=None,symbol='BTC-USD', end_date_str=None, period='1d', interval='1d')
new_vol_data
    

[*********************100%***********************]  1 of 1 completed


,date,negative_bert,neutral_bert,positive_bert,daily_tweets,MA7_Sentiment,MA20_Sentiment,MA50_Sentiment,BTC_price,BTC_trading_volume
0,2021-01-01,0.085275,0.489181,0.425545,12561.859509,NaN,NaN,NaN,29374.152344,40730301359
1,2021-01-02,0.143365,0.439075,0.417560,30124.352145,NaN,NaN,NaN,32127.267578,67865420765
2,2021-01-03,0.148220,0.432023,0.419757,26167.331282,NaN,NaN,NaN,32782.023438,78665235202
3,2021-01-04,0.101558,0.507526,0.390916,18041.199084,NaN,NaN,NaN,31971.914062,81163475344
4,2021-01-05,0.088530,0.517456,0.394014,17473.551568,NaN,NaN,NaN,33992.429688,67547324782
...,...,...,...,...,...,...,...,...,...,...
798,2023-03-10,0.122003,0.496804,0.381193,23029.804160,0.365374,0.357392,0.361947,20187.244141,39578257695
799,2023-03-11,0.118417,0.483141,0.398441,21168.432091,0.371627,0.359184,0.362839,20632.410156,30180288176
800,2023-03-12,0.137051,0.479557,0.383392,18621.077853,0.372813,0.359771,0.362830,22163.949219,29279035521
801,2023-03-13,0.126823,0.496415,0.376762,24769.727538,0.375752,0.360913,0.362850,24197.533203,49466362688


In [38]:
new_vol_data.to_csv("~/Code/giadapi/crypto/streamlit_data/master_vol_data.csv")

In [39]:
fully_combined.to_csv("~/Code/giadapi/crypto/streamlit_data/master_tweet_data.csv")